# Project: Wrangle and Analyze Data

In [4]:
import json
import os
import pandas as pd
import requests
import tweepy

## Gather Data

In [2]:
# Contains tweets with extra info already parsed from text
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# Predictions of dog breeds
url_image_predictions = ('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/'
                         '599fd2ad_image-predictions/image-predictions.tsv')
r = requests.get(url_image_predictions)
image_predictions = r.content

In [ ]:
# Twitter API for more data (save to local file)
consumer_key = 'YOUR CONSUMER KEY'
consumer_secret = 'YOUR CONSUMER SECRET'
access_token = 'YOUR ACCESS TOKEN'
access_secret = 'YOUR ACCESS SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [9]:
# Store data on tweet_json.txt
with open('tweet_json.txt', 'w') as outfile: 
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id)
            # Convert to JSON and write
            json.dump(tweet._json, outfile)
            outfile.write("\n")
            
        except:  # could be the case the tweet was deleted
            print('ERROR: Error on tweet id {}'.format(tweet_id))
            e = os.sys.exc_info()[0]

ERROR: Error on tweet id 888202515573088257
ERROR: Error on tweet id 873697596434513921
ERROR: Error on tweet id 869988702071779329
ERROR: Error on tweet id 866816280283807744
ERROR: Error on tweet id 861769973181624320
ERROR: Error on tweet id 842892208864923648
ERROR: Error on tweet id 827228250799742977
ERROR: Error on tweet id 802247111496568832
ERROR: Error on tweet id 775096608509886464
Rate limit reached. Sleeping for: 715
Rate limit reached. Sleeping for: 754


In [60]:
# Save only certain tweet elements in dataframe
elements_to_save = ['id', 'favorite_count', 'retweet_count']
# Later convert list to dataframe
data = []

with open('tweet_json.txt', 'r') as readfile: 
    # Read in JSON line and convert to dict
    tweet_json = readfile.readline()
    
    # Read line by line into DataFrame
    while tweet_json:
        tweet_dict = json.loads(tweet_json)
        # Create a smaller dict
        data_row = dict((k, tweet_dict[k]) for k in elements_to_save)
        data.append(data_row)
        
        # Read in JSON line and convert to dict
        tweet_json = readfile.readline()
        
            
    
df_tweet_info = pd.DataFrame.from_dict(data)